In [13]:
import pandas as pd
import re
import pyarrow

In [112]:
%pip install textblob
%pip install nltk
%pip install gensim
%pip install pyLDAvis
%pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 12.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 45.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 49.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.6 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.5/379.5 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: fi

In [113]:
import time
import math
import re
# import textblob
# from textblob import TextBlob
import pandas as pd
import numpy as np

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import string
import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess
pyLDAvis.enable_notebook()

In [132]:
# Defining stopwords
nltk.download('stopwords')

from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')).union(STOPWORDS)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Imports the Google Cloud client library
from google.cloud import storage
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "nlp_final_project_kshitijm"

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_final_project_kshitijm connected.


In [16]:
df = pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data1.csv', lineterminator='\n')

In [17]:
df.head()

,Unnamed: 0,url,date,language,title,text,cleaned_text
0,0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effi...
1,1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,en,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and S...
2,2,http://www.dataweek.co.za/12835r,2021-03-26,en,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...","Forget ML, AI and Industry 4.0 – obsolescence..."
3,3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71% of Smartphones Sold G...
4,4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,en,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,Olympus to Support Endoscopic AI Diagnosis Ed...


## First Tokenizing the titles to see the breadth of articles in the corpus

In [21]:
pd.options.display.max_colwidth=None
df['title'].sample(10)

139737                                  Home Security Solutions Market to Expand Radically; Emergence of AI and Machine Learning Technologies to Favor Growth, states Fortune Business Insights™ | Benzinga
137548    Artificial Intelligence as a Service Market Projected to Reach USD XX.XXBn By 2028:  Microsoft Corporation,  Google LLC,  Amazon Web Services Inc.,  IBM Corporation,  Fujitsu Ltd. – The Galleon
139320                            Apixio's New Apicare AuthAdvisor Leverages Machine Learning, Predictive Decision-Making to Automate Approvals & Reduce Manual Workload by more than 50% | News | wfmz.com
75097                                                                                                      ‘I'm getting way too much credit…': OpenAI CEO Sam Altman on company's success - Hindustan Times
104943                                                                                                                                          Spotify’s New AI DJ Thinks It Knows Whic

In [29]:
df['title'].sample(10)

1470                                                                                                                     Levi’s will begin using AI models to increase diversity, sustainability
140380                                                                                                            emtelligent Releases New Apps, Including AI Concordance Reporting at RSNA 2022
198669    Comprehensive Analysis of Global AI in Fintech Market with Current and Future Business Outlook | Microsoft (US), Google (California, US), Salesforce.com (US), IBM (US) – TechnoWeekly
1334                                                                                                       IA, chatGPT blocked by the Privacy Guarantor-breakinglatest.news-Breaking Latest News
17605                                                                                                                      What is formative AI and why should businesses care? -  Channel Asia 
107013                             

### We want to remove bylines from the titles, which indicate news sources. Rather, we can store it in another column to just focus on the title content
Most bylines start with [ - ],[ | ], [; ] 

In [99]:
# Cleaning titles
df['title']=df['title'].apply(lambda x: re.sub(r'(?:[\n]+)','',str(x)))
df['title']=df['title'].apply(lambda x: re.sub(r'(?:[\t]+)','',str(x)))

In [100]:
def find_source(x):
    pat_source=r'(?:\s\-[\s]+)([\w\W\d\D\s\S]*)|(?:\s\|[\s]+)([\w\W\d\D\s\S]*)|(?:\;[\s]+)([\w\W\d\D\s\S]*)|(?:\s\–[\s]+)([\w\W\d\D\s\S]*)'
    matches = re.findall(pat_source, x)
    if matches:
        src = ''.join(matches[-1]).strip()
        return src
    else:
        return ""

In [101]:
df['article_source']=df['title'].apply(lambda x: find_source(str(x)))

In [107]:
df[['title','article_source']].sample(10)

,title,article_source
44876,"Machine Learning in Finance Market Emerging Trends, Size, Share, Regional Outlook by 2027 – Atlantic Financial Management",Atlantic Financial Management
5352,APA AI set to replace managers' workloads - APA,APA
27792,Oxipit Closes $4.9M Funding Round To Advance AI Autonomy in Medical Imaging,
94008,"Adept, a startup training AI to use existing software and APIs, raises $350M – FocusTechnica",FocusTechnica
177919,Baidu to implement ChatGPT-like Ernie Bot chatbot from March | The Hill,The Hill
43899,Microsoft technology realizes “Minecraft” AI automatic construction- Hong Kong mobile game network GameApps.hk-breakinglatest.news-Breaking Latest News,
83263,Genesys Positioned as a Major Player in IDC MarketScape: Worldwide Conversational AI Software Platforms for Customer Service 2021 Vendor Assessment,
54256,Robots run the show as Swiss radio tests AI voices for a day,
188132,"SK hynix Develops PIM, Next-Generation AI Accelerator",
175102,Burns Night-in: Scotland celebrates the bard at home - Evening Express,Evening Express


In [108]:
df['clean_title']=df['title'].apply(lambda x: re.sub(pat_source,'',str(x)))
df[['title','clean_title','article_source']].sample(10)

,title,clean_title,article_source
98021,"Innovaccer Unveils Sara, Healthcare's First Self-Serve Conversational AI Assistant at HIMSS23","Innovaccer Unveils Sara, Healthcare's First Self-Serve Conversational AI Assistant at HIMSS23",
83470,Microsoft’s new Bing and Edge hands-on: Surprisingly well-integrated AI – The Hamden Journal,Microsoft’s new Bing and Edge hands-on: Surprisingly well-integrated AI,The Hamden Journal
17720,Investors’ query: Can Google answer Microsoft’s AI threat? – KSNF/KODE | FourStatesHomepage.com,Investors’ query: Can Google answer Microsoft’s AI threat?,KSNF/KODE | FourStatesHomepage.com
144130,Sonoma County to Add Artificial Intelligence for Fires – NBC Bay Area,Sonoma County to Add Artificial Intelligence for Fires,NBC Bay Area
166365,"Artificial Intelligence in the Education Sector Market to Witness Huge Growth by 2026 | IBM, Quantum adaptive learning, ALKES, Dreambox learning – The Courier",Artificial Intelligence in the Education Sector Market to Witness Huge Growth by 2026,"IBM, Quantum adaptive learning, ALKES, Dreambox learning – The Courier"
123989,Meta's new AI tool can predict protein shapes | Popular Science,Meta's new AI tool can predict protein shapes,Popular Science
145383,"Stage Fund Acquires Worklete, an AI Based Data Platform for Injury Prevention | News | wfmz.com","Stage Fund Acquires Worklete, an AI Based Data Platform for Injury Prevention",News | wfmz.com
30015,"Investing in data, AI is crucial for country","Investing in data, AI is crucial for country",
140628,"Buy Upstart, An AI Leader In Lending (NASDAQ:UPST) | Seeking Alpha","Buy Upstart, An AI Leader In Lending (NASDAQ:UPST)",Seeking Alpha
196439,"Reddit will charge companies for API access, citing AI training concerns – The Hamden Journal","Reddit will charge companies for API access, citing AI training concerns",The Hamden Journal


### Tokenizing Text

In [135]:
def create_tokens(sentence):
    list_tokens=list(gensim.utils.simple_preprocess(str(sentence), deacc=True, min_len=2, max_len=15))  # deacc=True removes punctuations
    list_tokens = [word for word in list_tokens if word not in stop_words]
    return list_tokens

In [136]:
df['title_tokens']=df['clean_title'].apply(lambda x: create_tokens(x))

In [144]:
df[['title','clean_title','title_tokens']].head(10)

,title,clean_title,title_tokens
0,Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online,Artificial intelligence improves parking efficiency in Chinese cities,"[artificial, intelligence, improves, parking, efficiency, chinese, cities]"
1,Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament,Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot,"[children, autism, saw, learning, social, skills, boosted, playing, ai, robot]"
2,"Forget ML, AI and Industry 4.0 – obsolescence should be your focus - 26 February 2021 - Test & Rework Solutions - Dataweek","Forget ML, AI and Industry 4.0","[forget, ml, ai, industry]"
3,Strategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered – Consumer Electronics Net,Strategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered,"[strategy, analytics, smartphones, sold, globally, ai, powered]"
4,Olympus to Support Endoscopic AI Diagnosis Education for Doctors in India and to Launch AI Diagnostic Support Application | | IT Business Net,Olympus to Support Endoscopic AI Diagnosis Education for Doctors in India and to Launch AI Diagnostic Support Application,"[olympus, support, endoscopic, ai, diagnosis, education, doctors, india, launch, ai, diagnostic, support, application]"
5,Cr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR) | Comtex SmarTrend,Cr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR),"[cr, bard, returned, smartrend, recommendation, bcr]"
6,From the Bard to broadcaster: Stratford Festival builds new identity with streamer | National Entertainment | pentictonherald.ca,From the Bard to broadcaster: Stratford Festival builds new identity with streamer,"[bard, broadcaster, stratford, festival, builds, new, identity, streamer]"
7,MulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision – Consumer Electronics Net,MulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision,"[multicoreware, ceva, trusted, partner, imaging, vision]"
8,Applitools Visual AI Reaches One Billion Images Analyzed | | IT Business Net,Applitools Visual AI Reaches One Billion Images Analyzed,"[applitools, visual, ai, reaches, billion, images, analyzed]"
9,"Artificial Intelligence In Behavioral And Mental Health Care Market to Witness Astonishing Growth by 2026 Focusing on Leading Players – AdvancedMD , Cerner , Core Solutions , Credible Behavioral Health – 3w Market News Reports",Artificial Intelligence In Behavioral And Mental Health Care Market to Witness Astonishing Growth by 2026 Focusing on Leading Players,"[artificial, intelligence, behavioral, mental, health, care, market, witness, astonishing, growth, focusing, leading, players]"


In [149]:
import itertools
all_title_tokens=list(itertools.chain(*df['title_tokens']))
all_title_tokens[0:10]

['artificial',
 'intelligence',
 'improves',
 'parking',
 'efficiency',
 'chinese',
 'cities',
 'children',
 'autism',
 'saw']

In [153]:
fdist_comm_lanc = nltk.FreqDist(all_title_tokens)
fdist_comm_lanc.most_common(20)

[('ai', 135331),
 ('intelligence', 37445),
 ('artificial', 36356),
 ('market', 27447),
 ('chatgpt', 17457),
 ('new', 15290),
 ('data', 14627),
 ('learning', 13158),
 ('global', 12440),
 ('machine', 12251),
 ('google', 9907),
 ('growth', 8444),
 ('platform', 8334),
 ('microsoft', 8194),
 ('analysis', 7946),
 ('launches', 7860),
 ('technology', 7753),
 ('research', 6279),
 ('industry', 6221),
 ('announces', 5906)]

#### From an initial view of the tokens, we can see that most of the articles revolve around AI, Data Science

In [154]:
df.to_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data2.csv')